In [ ]:
import pytorch_tabnet
from pytorch_tabnet.tab_model import TabNetClassifier
import requests as r
import pandas as pd
import datetime
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import pickle

Der Datensatz

In [ ]:
play_data = pd.read_csv(
    '/Users/michelbauer/Downloads/data_final(1).csv'
)

Aufteilung in Trainings-, Validierung, und Testdatensatz

In [ ]:

import numpy as np
from sklearn.model_selection import train_test_split


unique_game_ids = play_data['GameID'].unique()


train_game_ids, remaining_game_ids = train_test_split(unique_game_ids, test_size=0.2, random_state=42)
train_game_ids, valid_game_ids = train_test_split(train_game_ids, test_size=0.2, random_state=42)
test_game_ids =remaining_game_ids

# Aufteilen des play_data-Datensatzes basierend auf den GameIDs
train_df = play_data[play_data['GameID'].isin(train_game_ids)]
valid_df = play_data[play_data['GameID'].isin(valid_game_ids)]
test_df = play_data[play_data['GameID'].isin(test_game_ids)]





In [ ]:
train_game_ids = set(train_df['GameID'])
valid_game_ids = set(valid_df['GameID'])
test_game_ids = set(test_df['GameID'])

intersection_train_valid = train_game_ids.intersection(valid_game_ids)
intersection_train_test = train_game_ids.intersection(test_game_ids)
intersection_valid_test = valid_game_ids.intersection(test_game_ids)

if len(intersection_train_valid) == 0 and len(intersection_train_test) == 0 and len(intersection_valid_test) == 0:
    print("Keine gemeinsamen GameIDs zwischen train_df, valid_df und test_df.")
else:
    print("Es gibt gemeinsame GameIDs zwischen train_df, valid_df und test_df.")

XGBoost

In [ ]:
from sklearn.metrics import confusion_matrix
import xgboost as xgb

xgb_model = xgb.XGBClassifier(eval_metric=['error', 'auc'], early_stopping_rounds=15)

y_train_xgb = train_df['Winner']
y_valid_xgb = valid_df['Winner']
y_test_xgb = test_df['Winner']
X_train_xgb = train_df.drop('Winner', axis=1)
X_valid_xgb = valid_df.drop('Winner', axis=1)
X_test_xgb = test_df.drop('Winner', axis=1)


y_valid_xgb = y_valid_xgb.astype(int)
y_train_xgb = y_train_xgb.astype(int)
y_test_xgb = y_test_xgb.astype(int)
cardinality_cols = ['GameID','type_text', 'team_play', 'start_down', 'end_down', 'season', 'week', 'Home_teamname', 'Away_teamname', 'leading_Team']
numerical_cols = ['scoring_game','point_difference','awayScore', 'homeScore', 'scoreValue', 'statYardage', 'quarter', 'clock', 'start_distance', 'start_yardlinie', 'start_yardsToEndzone', 'end_distance', 'end_yardlinie', 'end_yardsToEndzone', 'elo_score_away', 'elo_score_home', 'attendence', 'time_remaining']


scaler = StandardScaler()
X_train_numerical_xgb = scaler.fit_transform(X_train_xgb[numerical_cols])
X_valid_numerical_xgb = scaler.transform(X_valid_xgb[numerical_cols])
X_test_numerical_xgb = scaler.transform(X_test_xgb[numerical_cols])

encoder = OneHotEncoder(handle_unknown='ignore')
X_train_categorical_xgb = encoder.fit_transform(X_train_xgb[cardinality_cols]).toarray()
X_valid_categorical_xgb = encoder.transform(X_valid_xgb[cardinality_cols]).toarray()
X_test_categorical_xgb = encoder.transform(X_test_xgb[cardinality_cols]).toarray()


X_train_processed_xgb = np.concatenate([X_train_numerical_xgb, X_train_categorical_xgb], axis=1)
X_valid_processed_xgb = np.concatenate([X_valid_numerical_xgb, X_valid_categorical_xgb], axis=1)
X_test_processed_xgb = np.concatenate([X_test_numerical_xgb, X_test_categorical_xgb], axis=1)



xgb_model.fit(
    X=X_train_processed_xgb,
    y=y_train_xgb,
    eval_set=[(X_valid_processed_xgb, y_valid_xgb)],
    verbose=True
)



preds_test_xgb = xgb_model.predict(X_test_processed_xgb)


accuracy_test_xgb = accuracy_score(y_test_xgb, preds_test_xgb)
precision_test_xgb = precision_score(y_test_xgb, preds_test_xgb)
recall_test_xgb = recall_score(y_test_xgb, preds_test_xgb)
f1_test_xgb = f1_score(y_test_xgb, preds_test_xgb)
auc_test_xgb = roc_auc_score(y_test_xgb, preds_test_xgb)
cm_test_xgb = confusion_matrix(y_test_xgb, preds_test_xgb)
cm_str_test_xgb = np.array2string(cm_test_xgb, separator=', ')


print("\nTest Set Metrics:")
print("Accuracy:", accuracy_test_xgb)
print("Precision:", precision_test_xgb)
print("Recall:", recall_test_xgb)
print("F1 Score:", f1_test_xgb)
print("AUC:", auc_test_xgb)
print("Confusion Matrix:\n", cm_str_test_xgb)

TabNet

In [ ]:

tn = TabNetClassifier(verbose=1, seed=42)

y_train_tn = train_df['Winner']
y_valid_tn = valid_df['Winner']
y_test_tn = test_df['Winner']
X_train_tn = train_df.drop('Winner', axis=1)
X_valid_tn = valid_df.drop('Winner', axis=1)
X_test_tn = test_df.drop('Winner', axis=1)

y_valid_tn = y_valid_tn.astype(int)
y_train_tn = y_train_tn.astype(int)
y_test_tn = y_test_tn.astype(int)
cardinality_cols = ['type_text', 'team_play', 'start_down', 'end_down', 'season', 'week', 'Home_teamname', 'Away_teamname', 'leading_Team']
numerical_cols = ['GameID','scoring_game','point_difference','awayScore', 'homeScore', 'scoreValue', 'statYardage', 'quarter', 'clock', 'start_distance', 'start_yardlinie', 'start_yardsToEndzone', 'end_distance', 'end_yardlinie', 'end_yardsToEndzone', 'elo_score_away', 'elo_score_home', 'attendence', 'time_remaining']

scaler = StandardScaler()
X_train_numerical_tn = scaler.fit_transform(X_train_tn[numerical_cols])
X_valid_numerical_tn = scaler.transform(X_valid_tn[numerical_cols])
X_test_numerical_tn = scaler.transform(X_test_tn[numerical_cols])


encoder = OneHotEncoder(handle_unknown='ignore')
X_train_categorical_tn = encoder.fit_transform(X_train_tn[cardinality_cols]).toarray()
X_valid_categorical_tn = encoder.transform(X_valid_tn[cardinality_cols]).toarray()
X_test_categorical_tn = encoder.transform(X_test_tn[cardinality_cols]).toarray()


X_train_processed_tn = np.concatenate([X_train_numerical_tn, X_train_categorical_tn], axis=1)
X_valid_processed_tn = np.concatenate([X_valid_numerical_tn, X_valid_categorical_tn], axis=1)
X_test_processed_tn = np.concatenate([X_test_numerical_tn, X_test_categorical_tn], axis=1)


tn.fit(
    X_train=X_train_processed_tn,
    y_train=y_train_tn,
    patience=30,
    max_epochs=32,
    eval_set=[(X_valid_processed_tn, y_valid_tn)],
    eval_metric=['accuracy', 'auc'],
    batch_size=32
)


preds_test_tn = tn.predict(X_test_processed_tn)

accuracy_test_tn = accuracy_score(y_test_tn, preds_test_tn)
precision_test_tn = precision_score(y_test_tn, preds_test_tn)
recall_test_tn = recall_score(y_test_tn, preds_test_tn)
f1_test_tn = f1_score(y_test_tn, preds_test_tn)
auc_test_tn = roc_auc_score(y_test_tn, preds_test_tn)
cm_test_tn = confusion_matrix(y_test_tn, preds_test_tn)
cm_str_test_tn = np.array2string(cm_test_tn, separator=', ')

print("Test Accuracy: {:.2f}".format(accuracy_test_tn))
print("Test Precision: {:.2f}".format(precision_test_tn))
print("Test Recall: {:.2f}".format(recall_test_tn))
print("Test F1 Score: {:.2f}".format(f1_test_tn))
print('Test Confusion Matrix: ', cm_str_test_tn)

Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
train_accs = []
valid_accs = []




n_estimators = [10, 50, 100, 200, 300, 400, 500]
y_train_rf = train_df['Winner']
y_valid_rf = valid_df['Winner']
y_test_rf = test_df['Winner']
X_train_rf = train_df.drop('Winner', axis=1)
X_valid_rf = valid_df.drop('Winner', axis=1)
X_test_rf = test_df.drop('Winner', axis=1)


y_valid_rf = y_valid_rf.astype(int)
y_train_rf = y_train_rf.astype(int)
y_test_rf = y_test_rf.astype(int)
cardinality_cols = ['GameID','type_text', 'team_play', 'start_down', 'end_down', 'season', 'week', 'Home_teamname', 'Away_teamname', 'leading_Team']
numerical_cols = ['scoring_game','point_difference','awayScore', 'homeScore', 'scoreValue', 'statYardage', 'quarter', 'clock', 'start_distance', 'start_yardlinie', 'start_yardsToEndzone', 'end_distance', 'end_yardlinie', 'end_yardsToEndzone', 'elo_score_away', 'elo_score_home', 'attendence', 'time_remaining']


scaler = StandardScaler()
X_train_numerical_rf = scaler.fit_transform(X_train_rf[numerical_cols])
X_valid_numerical_rf = scaler.transform(X_valid_rf[numerical_cols])
X_test_numerical_rf = scaler.transform(X_test_rf[numerical_cols])


encoder = OneHotEncoder(handle_unknown='ignore')
X_train_categorical_rf = encoder.fit_transform(X_train_rf[cardinality_cols]).toarray()
X_valid_categorical_rf = encoder.transform(X_valid_rf[cardinality_cols]).toarray()
X_test_categorical_rf = encoder.transform(X_test_rf[cardinality_cols]).toarray()


X_train_processed_rf = np.concatenate([X_train_numerical_rf, X_train_categorical_rf], axis=1)
X_valid_processed_rf = np.concatenate([X_valid_numerical_rf, X_valid_categorical_rf], axis=1)
X_test_processed_rf = np.concatenate([X_test_numerical_rf, X_test_categorical_rf], axis=1)


for n in n_estimators:
    
    rf_model = RandomForestClassifier(n_estimators=n, random_state=0)
    rf_model.fit(X_train_processed_rf, y_train_rf)

    
    preds_train_rf = rf_model.predict(X_train_processed_rf)
    preds_valid_rf = rf_model.predict(X_valid_processed_rf)

    
    train_acc = accuracy_score(y_train_rf, preds_train_rf)
    valid_acc = accuracy_score(y_valid_rf, preds_valid_rf)

    
    train_accs.append(train_acc)
    valid_accs.append(valid_acc)


plt.plot(n_estimators, train_accs, label='Training Accuracy')
plt.plot(n_estimators, valid_accs, label='Validation Accuracy')
plt.xlabel('Number of Trees')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Validation Curve: Random Forest')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier


rf_model = RandomForestClassifier(n_estimators=100, random_state=0)


y_train_rf = train_df['Winner']
y_valid_rf = valid_df['Winner']
y_test_rf = test_df['Winner']
X_train_rf = train_df.drop('Winner', axis=1)
X_valid_rf = valid_df.drop('Winner', axis=1)
X_test_rf = test_df.drop('Winner', axis=1)


y_valid_rf = y_valid_rf.astype(int)
y_train_rf = y_train_rf.astype(int)
y_test_rf = y_test_rf.astype(int)


cardinality_cols = ['GameID','type_text', 'team_play', 'start_down', 'end_down', 'season', 'week', 'Home_teamname', 'Away_teamname', 'leading_Team']
numerical_cols = ['scoring_game','point_difference','awayScore', 'homeScore', 'scoreValue', 'statYardage', 'quarter', 'clock', 'start_distance', 'start_yardlinie', 'start_yardsToEndzone', 'end_distance', 'end_yardlinie', 'end_yardsToEndzone', 'elo_score_away', 'elo_score_home', 'attendence', 'time_remaining']



scaler = StandardScaler()
X_train_numerical_rf = scaler.fit_transform(X_train_rf[numerical_cols])
X_valid_numerical_rf = scaler.transform(X_valid_rf[numerical_cols])
X_test_numerical_rf = scaler.transform(X_test_rf[numerical_cols])


encoder = OneHotEncoder(handle_unknown='ignore')
X_train_categorical_rf = encoder.fit_transform(X_train_rf[cardinality_cols]).toarray()
X_valid_categorical_rf = encoder.transform(X_valid_rf[cardinality_cols]).toarray()
X_test_categorical_rf = encoder.transform(X_test_rf[cardinality_cols]).toarray()


X_train_processed_rf = np.concatenate([X_train_numerical_rf, X_train_categorical_rf], axis=1)
X_valid_processed_rf = np.concatenate([X_valid_numerical_rf, X_valid_categorical_rf], axis=1)
X_test_processed_rf = np.concatenate([X_test_numerical_rf, X_test_categorical_rf], axis=1)


rf_model.fit(X=X_train_processed_rf,
              y=y_train_rf)


preds_test_rf = rf_model.predict(X_test_processed_rf)


accuracy_test_rf = accuracy_score(y_test_rf, preds_test_rf)
precision_test_rf = precision_score(y_test_rf, preds_test_rf)
recall_test_rf = recall_score(y_test_rf, preds_test_rf)
f1_test_rf = f1_score(y_test_rf, preds_test_rf)
auc_test_rf = roc_auc_score(y_test_rf, preds_test_rf)
cm_test_rf = confusion_matrix(y_test_rf, preds_test_rf)
cm_str_test_rf = np.array2string(cm_test_rf, separator=', ')

print("\nTest Set Metrics:")
print("Accuracy:", accuracy_test_rf)
print("Precision:", precision_test_rf)
print("Recall:", recall_test_rf)
print("F1 Score:", f1_test_rf)
print("AUC:", auc_test_rf)
print("Confusion Matrix:\n", cm_str_test_rf)